In [2]:
import torch
import PIL.Image as Image
from collections import OrderedDict
import torchvision.transforms.v2 as tr

In [4]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self):
        super().__init__()
        self.f = open('data/dog-breed-identification/labels.csv')
        self.f = self.f.read().split()[1:]
        for i in range(len(self.f)):
            self.f[i] = self.f[i].split(',')
        self.uniqs = list(set([self.f[i][1] for i in range(len(self.f))])) # все породы собак
        self.categories = OrderedDict([(self.uniqs[i], i) for i in range(len(self.uniqs))]) # все породы собак с их индексами
        self.transforms = tr.Compose([
            tr.Resize((500, 500)),
            tr.ToImage(),
            tr.ToDtype(torch.float32, scale=True)
        ])

    def __getitem__(self, index):
        return (self.transforms(Image.open(f'data/dog-breed-identification/train/{self.f[index][0]}.jpg').convert('RGB')), self.categories[self.f[index][1]])

    def __len__(self):
        return len(self.f)
    

class Identifikator(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.func = torch.nn.Sequential(
            torch.nn.Conv2d(3, 25, 3, 1, 1, bias=False),
            torch.nn.BatchNorm2d(10),
            torch.nn.MaxPool2d(5),
            torch.nn.Conv2d(25, 100, 3, 1, 1, bias=False),
            torch.nn.BatchNorm2d(100),
            torch.nn.MaxPool2d(2),
            torch.nn.Linear(2500, 1000),
            torch.nn.ReLU(True),
            torch.nn.Linear(1000, 120)
        )

    def forward(self, x):
        return self.func(x)


data = Dataset()
train_dataset, test_dataset = torch.utils.data.random_split(data, [0.75, 0.25])
train_data = torch.utils.data.DataLoader(train_dataset, 10, True)
test_data = torch.utils.data.DataLoader(test_dataset, 10, True)

model=Identifikator()
model.train()


Identifikator(
  (func): Sequential(
    (0): Conv2d(3, 25, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): MaxPool2d(kernel_size=5, stride=5, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(25, 100, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Linear(in_features=2500, out_features=1000, bias=True)
    (7): ReLU(inplace=True)
    (8): Linear(in_features=1000, out_features=120, bias=True)
  )
)